# Building Concepts 

In [1]:
!pip install selenium
!pip install msedge-selenium-tools==3.141.3
!pip install webdriver-manager
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 995 kB 5.3 MB/s 
     |████████████████████████████████| 384 kB 71.0 MB/s 
     |████████████████████████████████| 140 kB 67.1 MB/s 
     |████████████████████████████████| 58 kB 5.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.12 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 904 kB 5.3 MB/s 
  Attempting uninstall: selenium
    Found existing installation

In [2]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver # Firefox & Chrome
from msedge.selenium_tools import Edge, EdgeOptions # Edge
from webdriver_manager.chrome import ChromeDriverManager

Run Link Using Webdriver

In [3]:
url = 'https://www.qatar2022.qa/book/en/search'

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_driver = webdriver.Chrome('chromedriver',options=chrome_options)

#https://sites.google.com/a/chromium.org/chromedriver/home
#edge_driver.get(url)
chrome_driver.get(url)
#firefox_drive.get(url)

soup = BeautifulSoup(chrome_driver.page_source, 'html.parser')

Extract Collection

In [10]:
results = soup.find_all('div', {'class': 'hotel-listing__item'})
#item_content = str(results[0]).split('class="hotel-listing__item-content">')[1]
item_content = results[0].find('div', 'hotel-listing__item-content')
item_availability = item_content.find('span', 'hotel-listing__item-price-burgundy').text
item_name = item_content.find('span', 'hotel-listing__item-name').text
item_link = item_content.find('a', 'hotel-listing__item-link').get('href')
print('Items Extracted:', len(results))
print('Sample: ')
print(item_availability)
print(item_name)
print(item_link)

Items Extracted: 10
Sample: 
Available
Fan Village Qetaifan Island North
https://www.qatar2022.qa/book/en/detail/57992/Fan-Village-Qetaifan-Island-North


# Page Numbers

In [13]:
pagnation = soup.find_all('div', {'class': 'pagination'})
#pagnation = item_content.find('div', 'pagination')
count_pg_no_btn = pagnation[0].find_all('button')
print('Pagenation Buttons ({}): '.format(len(count_pg_no_btn)), count_pg_no_btn)
disabled_btn = pagnation[0].find_all('button', {'disabled': 'disabled'})
last_pg_btn = count_pg_no_btn[count_pg_no_btn.index(disabled_btn[-1])+1]
pg_number_max = last_pg_btn.find('a').text
print('Maximum Page Number: ',pg_number_max)

<div class="pagCentered pagination" data-gtmpage="1" data-v-356edd9b="" data-v-3645943b=""><ul data-v-356edd9b=""><button data-v-356edd9b="" disabled="disabled"><li class="pagination__prev"><a>Previous</a></li></button><button data-v-356edd9b=""><li class="-active"><a>1</a></li></button><button data-v-356edd9b=""><li><a>2</a></li></button><button data-v-356edd9b=""><li><a>3</a></li></button><button data-v-356edd9b=""><li><a>4</a></li></button><button data-v-356edd9b=""><li><a>5</a></li></button><button data-v-356edd9b=""><li><a>6</a></li></button><button data-v-356edd9b=""><li><a>7</a></li></button><button data-v-356edd9b="" disabled="disabled"><li>...</li></button><button data-v-356edd9b=""><li><a>61</a></li></button><button data-v-356edd9b=""><li class="pagination__next"><a>Next</a></li></button></ul></div>
Pagenation Buttons (11):  [<button data-v-356edd9b="" disabled="disabled"><li class="pagination__prev"><a>Previous</a></li></button>, <button data-v-356edd9b=""><li class="-active

In [15]:
# Figuring Button Click Page Change
print(pagnation[0])
#btn_pg = chrome_driver.find_element_by_css_selector('li.-active').click()
btn_pg = chrome_driver.find_element_by_link_text("2").click() # Pagenation not clickcable
print(btn_pg.text)

<div class="pagCentered pagination" data-gtmpage="1" data-v-356edd9b="" data-v-3645943b=""><ul data-v-356edd9b=""><button data-v-356edd9b="" disabled="disabled"><li class="pagination__prev"><a>Previous</a></li></button><button data-v-356edd9b=""><li class="-active"><a>1</a></li></button><button data-v-356edd9b=""><li><a>2</a></li></button><button data-v-356edd9b=""><li><a>3</a></li></button><button data-v-356edd9b=""><li><a>4</a></li></button><button data-v-356edd9b=""><li><a>5</a></li></button><button data-v-356edd9b=""><li><a>6</a></li></button><button data-v-356edd9b=""><li><a>7</a></li></button><button data-v-356edd9b="" disabled="disabled"><li>...</li></button><button data-v-356edd9b=""><li><a>61</a></li></button><button data-v-356edd9b=""><li class="pagination__next"><a>Next</a></li></button></ul></div>


ElementClickInterceptedException: ignored

# Complete Prototype

In [16]:
def extract_record(item):
  item_content = item.find('div', 'hotel-listing__item-content')
  try:
    item_availability = item_content.find('span', 'hotel-listing__item-price-burgundy').text
    item_name = item_content.find('span', 'hotel-listing__item-name').text
    item_link = item_content.find('a', 'hotel-listing__item-link').get('href')
  except:
    item_name = ''
    item_availability = ''
    item_link = ''

  result = (item_name, item_availability, item_link)

  return result

In [17]:
records = []
results = soup.find_all('div', {'class': 'hotel-listing__item'})

for item in results:
  record = extract_record(item)
  if record:
    records.append(record)
print(records)

[('Fan Village Qetaifan Island North', 'Available', 'https://www.qatar2022.qa/book/en/detail/57992/Fan-Village-Qetaifan-Island-North'), ('4340 Umm Besher Barwa Madinatna Cluster G', 'Available', 'https://www.qatar2022.qa/book/en/detail/57776/4340-Umm-Besher-Barwa-Madinatna-Cluster-G'), ('Fan Village Cabins Free Zone', 'Available', 'https://www.qatar2022.qa/book/en/detail/57833/Fan-Village-Cabins-Free-Zone'), ('Cruise Ship Hotel - MSC World Europa', 'Available', 'https://www.qatar2022.qa/book/en/detail/57367/Cruise-Ship-Hotel---MSC-World-Europa'), ('4381 Barwa Barahat Al Janoub Cluster T', 'Available', 'https://www.qatar2022.qa/book/en/detail/57826/4381-Barwa-Barahat-Al-Janoub-Cluster-T'), ('3351 Al Kharayej Apartments', 'Available', 'https://www.qatar2022.qa/book/en/detail/57571/3351-Al-Kharayej-Apartments'), ('Fan Village Al Khor', 'Available', 'https://www.qatar2022.qa/book/en/detail/57875/Fan-Village-Al-Khor'), ('3234 Fereej Bin Mahmoud 23 Apartments', 'Available', 'https://www.qata

In [18]:
import pandas as pd
df = pd.DataFrame()

names, availabilities, links = [], [], []

for record in records:
  names.append(record[0])
  availabilities.append(record[1])
  links.append(record[2])

df['Item Name'] = names
df['Item Availability'] = availabilities
df['Item Link'] = links
df.head(len(df))

,Item Name,Item Availability,Item Link
0,Fan Village Qetaifan Island North,Available,https://www.qatar2022.qa/book/en/detail/57992/...
1,4340 Umm Besher Barwa Madinatna Cluster G,Available,https://www.qatar2022.qa/book/en/detail/57776/...
2,Fan Village Cabins Free Zone,Available,https://www.qatar2022.qa/book/en/detail/57833/...
3,Cruise Ship Hotel - MSC World Europa,Available,https://www.qatar2022.qa/book/en/detail/57367/...
4,4381 Barwa Barahat Al Janoub Cluster T,Available,https://www.qatar2022.qa/book/en/detail/57826/...
5,3351 Al Kharayej Apartments,Available,https://www.qatar2022.qa/book/en/detail/57571/...
6,Fan Village Al Khor,Available,https://www.qatar2022.qa/book/en/detail/57875/...
7,3234 Fereej Bin Mahmoud 23 Apartments,Available,https://www.qatar2022.qa/book/en/detail/58062/...
8,Sambook Ship Dhows,Available,https://www.qatar2022.qa/book/en/detail/58109/...
9,3310 Muaither 55 Villas,Available,https://www.qatar2022.qa/book/en/detail/57482/...


In [ ]:
df.to_excel('sample_hotel_names.xlsx')